# Utilities

In [1]:
import pandas as pd
import numpy as np
import re
import math
import pickle
import glob
import xgboost
import tsfresh
import os
import hockey_scraper
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from pprint import pprint
from sklearn import svm
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import export_graphviz
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score, r2_score, log_loss
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_context('notebook')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.5, 's' : 100, 'linewidths':0}
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pd.options.mode.chained_assignment = None  # default='warn'

# Variables
scrape = True

# Scrape

In [ ]:
if scrape:
    # Pbp data deposited in file - /Users/noiseuce/hockey_scraper_data/csvs/nhl_pbp20182019.csv
    # Shift data deposited in file - /Users/noiseuce/hockey_scraper_data/csvs/nhl_shifts20182019.csv
#     hockey_scraper.scrape_seasons([2018], True, docs_dir=True)
    hockey_scraper.scrape_seasons([2014, 2015, 2016, 2017], True, docs_dir=True)

# Reformat

In [ ]:
%%time
nhl_pbp = pd.read_csv('../data/hockey_scraper_data/csvs/nhl_pbp20182019.csv')
nhl_shifts = pd.read_csv('../data/hockey_scraper_data/csvs/nhl_shifts20182019.csv').drop('Unnamed: 0', axis=1)

In [ ]:
# Keep regular season games only
nhl_pbp['Date'] = pd.to_datetime(nhl_pbp['Date'])
nhl_pbp = nhl_pbp[nhl_pbp.Date <= pd.datetime(2019, 4, 6)]
nhl_shifts['Date'] = pd.to_datetime(nhl_shifts['Date'])
nhl_shifts = nhl_shifts[nhl_shifts.Date <= pd.datetime(2019, 4, 6)]

# For each game ID, get all players that played in it
player_games = nhl_shifts.sort_values(['Game_Id', 'Player']).drop_duplicates(subset=['Game_Id', 'Player'])[['Game_Id', 'Team', 'Player', 'Player_Id', 'Date']]

# For each game ID, get all players with goal(s) or assist(s)
player_games_points = nhl_pbp[nhl_pbp['Event'] == 'GOAL'][['Game_Id', 'p1_name', 'p1_ID', 'p2_name', 'p2_ID', 'p3_name', 'p3_ID']]
player_games_points = player_games_points.fillna(0)

In [ ]:
# Get goals scored per player in each games
# Keep game_id, player name and id, and number of goals
player_games_goals = player_games_points.groupby(['Game_Id', 'p1_name', 'p1_ID']).count().reset_index()
player_games_goals = player_games_goals.iloc[:,:4]
player_games_goals.columns = ['Game_Id', 'Player', 'Player_Id', 'Goal']

# Get first assists per player in each games
# Keep game_id, player name and id, and number of assists
player_games_first_assists = player_games_points.groupby(['Game_Id', 'p2_name', 'p2_ID']).count().reset_index()
player_games_first_assists = player_games_first_assists[player_games_first_assists['p2_name'] != 0]
player_games_first_assists = player_games_first_assists.iloc[:,:4]
player_games_first_assists.columns = ['Game_Id', 'Player', 'Player_Id', 'First_Assist']

# Get second assists per player in each games
# Keep game_id, player name and id, and number of assists
player_games_second_assists = player_games_points.groupby(['Game_Id', 'p3_name', 'p3_ID']).count().reset_index()
player_games_second_assists = player_games_second_assists[player_games_second_assists['p3_name'] != 0]
player_games_second_assists = player_games_second_assists.iloc[:,:4]
player_games_second_assists.columns = ['Game_Id', 'Player', 'Player_Id', 'Second_Assist']

In [ ]:
# Merge goals and first assist and second assist
player_logs = player_games.merge(player_games_goals, on=['Game_Id', 'Player', 'Player_Id'], how='outer')
player_logs = player_logs.merge(player_games_first_assists, on=['Game_Id', 'Player', 'Player_Id'], how='outer')
player_logs = player_logs.merge(player_games_second_assists, on=['Game_Id', 'Player', 'Player_Id'], how='outer')
player_logs = player_logs.fillna(0)

# Add total points
player_logs['Total_Points'] = player_logs[['Goal', 'First_Assist', 'Second_Assist']].sum(axis=1)
player_logs.head()